# Assignment 3: MNIST Points classification with various point-based architectures

In this assignment, we will implement point-based network with PyTorch. Unlike other methods based on voxelization or multi-view projection, point-based networks take as input raw point coordinates directly. The goal of this assignment is for you to learn the popular components of point-based netwoks -- Shared MLP (PointNet), ACNe/CNe block, Set abstraction layer (PointNet++) and point convolution. We have outlined TODOs in a way that they will sort of *force* you to read the code. Hopefully it's not too hard!

- **PointNet**: The very first version of point-based network.
- **CNe/ACNe**: Point cloud networks with a focus on global message passing -- (Attentive) context normalization. 
- **PointNet++**: A network that introduce the hierarchical structure and locality into point-based networks.
- **Point Convolution**: A network that defines the convolution for point clouds.


### What to do?
Walk through this notebook and edit so that it becomes your report. Most of the work would be in the separate Python files provided alongside this notebook. When you are submitting, simply zip your notebook and your python files all together and upload it to canvas. Importantly, do **NOT** include the data files, nor hidden ipython checkpoints in your zip file -- it'll make our hard drives expload.


### Due date
Deadline: 11/06/2023, 11:59 pm

## Input pipeline 
We will continue using our MNIST POINTS dataset loader that we implemented in our first assignment.

In [1]:
# Prepare the dataset MNISTPTS if you don't have. 
import numpy as np
from utils.mnist_helper import load_mnist, load_mnistpts, dump_mnistpts

data_mnist_dir = 'data' # directory saving raw MNIST image data.
data_mnistpts_dir = 'data_dump' # directory saving point cloud.

# Downloading and preprocessing MNIST dataset
for mode in ['train', 'test']:
    print(f'Processing {mode} set')
    dump_mnistpts(data_mnist_dir, data_mnistpts_dir, mode)

print('Done.')

Processing train set
Processing test set
MNIST found in data
Done.


In [2]:
## Get dataloader
from get_dataloader import get_dataloader
from get_config import get_config

config = get_config()

loader_tr, loader_va = get_dataloader(config, mode='train')
loader_te = get_dataloader(config, mode='test')[0]


loading train datasets.
Number of training samples: 48000
Number of valid samples: 12000
loading test datasets.
Number of test samples: 10000


## Training and testing networks

Roughtly, we focus on implementing different components in following networks. Please finish the TODOs in the corresponding python scripts. Note that some of the TODOs for PointNet++ and Point Convolutions overlap and they can be implemented identically.

- **FCNet**: We have finished it in assignment1. 
- **PointNet**: Using the shared MLP layer and implementing the framework.
- **CNe/ACNe**: (Attentive) context normalization layer.
- **PointNet++**: Set abstraction layer. (Includes k-nn implementation at `nn_pts.py`)
- **Point Convolution**: Point Convolution.



In [3]:
# Train FCNet which we have finished in assignment1.

from network import Network
from get_model import get_model

config.model = 'fcnet' # Specify the model that you would like to run. 
config.order_pts = True
model = get_model(config)
net = Network(model, config)

# Training on train and validation set.
net.train(loader_tr, loader_va)

# Testing on test set.
oa = net.test(loader_te)
print(f'Test OA: {oa.item()}')

The number of parameters in fcnet:     34.31k
Epoch:   0, loss_avg: 2.26321, val OA: 0.27692, best val OA: 0.27692
Epoch:   1, loss_avg: 1.95652, val OA: 0.37300, best val OA: 0.37300
Epoch:   2, loss_avg: 1.69345, val OA: 0.44892, best val OA: 0.44892
Epoch:   3, loss_avg: 1.54810, val OA: 0.48875, best val OA: 0.48875
Epoch:   4, loss_avg: 1.43566, val OA: 0.52250, best val OA: 0.52250
Epoch:   5, loss_avg: 1.34998, val OA: 0.55117, best val OA: 0.55117
Epoch:   6, loss_avg: 1.28386, val OA: 0.57542, best val OA: 0.57542
Epoch:   7, loss_avg: 1.23061, val OA: 0.59192, best val OA: 0.59192
Epoch:   8, loss_avg: 1.18501, val OA: 0.60075, best val OA: 0.60075
Epoch:   9, loss_avg: 1.14728, val OA: 0.61767, best val OA: 0.61767
Epoch:  10, loss_avg: 1.11256, val OA: 0.63333, best val OA: 0.63333
Epoch:  11, loss_avg: 1.07690, val OA: 0.63833, best val OA: 0.63833
Epoch:  12, loss_avg: 1.04672, val OA: 0.64267, best val OA: 0.64267
Epoch:  13, loss_avg: 1.02122, val OA: 0.65483, best val 

## ACNe and CNe

In [4]:
# Train CNe

from network import Network
from get_model import get_model

config.model = 'cne'
config.cn_opt = 'cn'
model = get_model(config)
net = Network(model, config)

# Training on train and validation set.
net.train(loader_tr, loader_va)

# Testing on test set.
oa = net.test(loader_te)
print(f'Test OA: {oa.item()}')


The number of parameters in cne:     37.04k
Epoch:   0, loss_avg: 0.91504, val OA: 0.82475, best val OA: 0.82475
Epoch:   1, loss_avg: 0.49894, val OA: 0.85500, best val OA: 0.85500
Epoch:   2, loss_avg: 0.42339, val OA: 0.86692, best val OA: 0.86692
Epoch:   3, loss_avg: 0.38659, val OA: 0.88908, best val OA: 0.88908
Epoch:   4, loss_avg: 0.34982, val OA: 0.89500, best val OA: 0.89500
Epoch:   5, loss_avg: 0.33327, val OA: 0.89767, best val OA: 0.89767
Epoch:   6, loss_avg: 0.32074, val OA: 0.90133, best val OA: 0.90133
Epoch:   7, loss_avg: 0.30847, val OA: 0.90225, best val OA: 0.90225
Epoch:   8, loss_avg: 0.30392, val OA: 0.90883, best val OA: 0.90883
Epoch:   9, loss_avg: 0.28756, val OA: 0.91058, best val OA: 0.91058
Epoch:  10, loss_avg: 0.28400, val OA: 0.89967, best val OA: 0.91058
Epoch:  11, loss_avg: 0.27992, val OA: 0.90575, best val OA: 0.91058
Epoch:  12, loss_avg: 0.27231, val OA: 0.91692, best val OA: 0.91692
Epoch:  13, loss_avg: 0.26894, val OA: 0.91775, best val OA

In [5]:
# Train ACNe via simply activating attention mechanism (i.e., cn ==> *a*cn)

from network import Network
from get_model import get_model

config.model = 'acne'
config.cn_opt = 'acn'

model = get_model(config)
net = Network(model, config)

# Training on train and validation set.
net.train(loader_tr, loader_va)

# Testing on test set.
oa = net.test(loader_te)
print(f'Test OA: {oa.item()}')


The number of parameters in acne:     37.11k
Epoch:   0, loss_avg: 0.75463, val OA: 0.86883, best val OA: 0.86883
Epoch:   1, loss_avg: 0.37352, val OA: 0.89975, best val OA: 0.89975
Epoch:   2, loss_avg: 0.32216, val OA: 0.90492, best val OA: 0.90492
Epoch:   3, loss_avg: 0.30031, val OA: 0.91925, best val OA: 0.91925
Epoch:   4, loss_avg: 0.27986, val OA: 0.91342, best val OA: 0.91925
Epoch:   5, loss_avg: 0.26677, val OA: 0.91392, best val OA: 0.91925
Epoch:   6, loss_avg: 0.26003, val OA: 0.92583, best val OA: 0.92583
Epoch:   7, loss_avg: 0.25145, val OA: 0.92408, best val OA: 0.92583
Epoch:   8, loss_avg: 0.25064, val OA: 0.92150, best val OA: 0.92583
Epoch:   9, loss_avg: 0.23911, val OA: 0.93217, best val OA: 0.93217
Epoch:  10, loss_avg: 0.23499, val OA: 0.92533, best val OA: 0.93217
Epoch:  11, loss_avg: 0.23378, val OA: 0.93117, best val OA: 0.93217
Epoch:  12, loss_avg: 0.23086, val OA: 0.93033, best val OA: 0.93217
Epoch:  13, loss_avg: 0.22411, val OA: 0.93492, best val O

## PointNet

In [6]:
# Train PointNet

from network import Network
from get_model import get_model

config.model = 'pointnet'
model = get_model(config)
net = Network(model, config)

# Training on train and validation set.
net.train(loader_tr, loader_va)

# Testing on test set.
oa = net.test(loader_te)
print(f'Test OA: {oa.item()}')

The number of parameters in pointnet:     37.04k
Epoch:   0, loss_avg: 0.91409, val OA: 0.81533, best val OA: 0.81533
Epoch:   1, loss_avg: 0.50138, val OA: 0.86175, best val OA: 0.86175
Epoch:   2, loss_avg: 0.41939, val OA: 0.88208, best val OA: 0.88208
Epoch:   3, loss_avg: 0.38266, val OA: 0.89325, best val OA: 0.89325
Epoch:   4, loss_avg: 0.35787, val OA: 0.90333, best val OA: 0.90333
Epoch:   5, loss_avg: 0.34213, val OA: 0.89975, best val OA: 0.90333
Epoch:   6, loss_avg: 0.33064, val OA: 0.90267, best val OA: 0.90333
Epoch:   7, loss_avg: 0.31927, val OA: 0.90800, best val OA: 0.90800
Epoch:   8, loss_avg: 0.30730, val OA: 0.91017, best val OA: 0.91017
Epoch:   9, loss_avg: 0.30415, val OA: 0.91433, best val OA: 0.91433
Epoch:  10, loss_avg: 0.29605, val OA: 0.91250, best val OA: 0.91433
Epoch:  11, loss_avg: 0.29124, val OA: 0.91492, best val OA: 0.91492
Epoch:  12, loss_avg: 0.28291, val OA: 0.91700, best val OA: 0.91700
Epoch:  13, loss_avg: 0.28081, val OA: 0.91683, best v

# PointNet++ 


In [7]:
# Train PointNet++

from network import Network
from get_model import get_model

config.model = 'pointnet2'
model = get_model(config)
net = Network(model, config)

# Training on train and validation set.
net.train(loader_tr, loader_va)

# Testing on test set.
oa = net.test(loader_te)
print(f'Test OA: {oa.item()}')

The number of parameters in pointnet2:     39.41k
Epoch:   0, loss_avg: 0.93575, val OA: 0.79333, best val OA: 0.79333
Epoch:   1, loss_avg: 0.58746, val OA: 0.82725, best val OA: 0.82725
Epoch:   2, loss_avg: 0.48754, val OA: 0.84833, best val OA: 0.84833
Epoch:   3, loss_avg: 0.42900, val OA: 0.86817, best val OA: 0.86817
Epoch:   4, loss_avg: 0.40367, val OA: 0.88617, best val OA: 0.88617
Epoch:   5, loss_avg: 0.37189, val OA: 0.89433, best val OA: 0.89433
Epoch:   6, loss_avg: 0.35075, val OA: 0.89783, best val OA: 0.89783
Epoch:   7, loss_avg: 0.33576, val OA: 0.89858, best val OA: 0.89858
Epoch:   8, loss_avg: 0.32607, val OA: 0.90542, best val OA: 0.90542
Epoch:   9, loss_avg: 0.31133, val OA: 0.90067, best val OA: 0.90542
Epoch:  10, loss_avg: 0.30428, val OA: 0.90692, best val OA: 0.90692
Epoch:  11, loss_avg: 0.29821, val OA: 0.91125, best val OA: 0.91125
Epoch:  12, loss_avg: 0.28844, val OA: 0.91392, best val OA: 0.91392
Epoch:  13, loss_avg: 0.28617, val OA: 0.91517, best 

## Point Convolution 

In [8]:
# Train Point Conv

from network import Network
from get_model import get_model

config.model = 'pointconv'
model = get_model(config)
net = Network(model, config)

# Training on train and validation set.
net.train(loader_tr, loader_va)

# Testing on test set.
oa = net.test(loader_te)
print(f'Test OA: {oa.item()}')

The number of parameters in pointconv:     40.12k
Epoch:   0, loss_avg: 1.18993, val OA: 0.70708, best val OA: 0.70708
Epoch:   1, loss_avg: 0.81364, val OA: 0.76142, best val OA: 0.76142
Epoch:   2, loss_avg: 0.69892, val OA: 0.79150, best val OA: 0.79150
Epoch:   3, loss_avg: 0.61711, val OA: 0.81700, best val OA: 0.81700
Epoch:   4, loss_avg: 0.56018, val OA: 0.83500, best val OA: 0.83500
Epoch:   5, loss_avg: 0.51732, val OA: 0.84733, best val OA: 0.84733
Epoch:   6, loss_avg: 0.48861, val OA: 0.86075, best val OA: 0.86075
Epoch:   7, loss_avg: 0.44536, val OA: 0.87050, best val OA: 0.87050
Epoch:   8, loss_avg: 0.42447, val OA: 0.88267, best val OA: 0.88267
Epoch:   9, loss_avg: 0.39739, val OA: 0.88008, best val OA: 0.88267
Epoch:  10, loss_avg: 0.37782, val OA: 0.89117, best val OA: 0.89117
Epoch:  11, loss_avg: 0.36577, val OA: 0.90108, best val OA: 0.90108
Epoch:  12, loss_avg: 0.35192, val OA: 0.89933, best val OA: 0.90108
Epoch:  13, loss_avg: 0.34608, val OA: 0.89842, best 

## Report (10 Pts)

(5 points) Please briefly discuss your findings. A quick discussion on how you find the validation performance, the convergence, and the efficiency of each method to be would suffice. 

(5 points) Also, is the above comparison fair? Is there anything that should be considered when comparing these methods?